In [1]:
#Load the libraries

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle
from bs4 import BeautifulSoup
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')

['movie-recommendation-system-indian-movies-df']


In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
dataset = pd.read_csv('../input/movie-recommendation-system-indian-movies-df/reviews.txt',sep = '\t', names =['sentiment','review'])

In [4]:
imdb_data = pd.read_csv('../input/movie-recommendation-system-indian-movies-df/IMDB Dataset.csv/IMDB Dataset.csv')
imdb_data.loc[:, 'sentiment'] = imdb_data.loc[:, 'sentiment'].map({'positive':1, 'negative':0})

In [5]:
#concating the 2 dataset and keeping only unique values
imdb_data = pd.concat([imdb_data,dataset])
imdb_data = imdb_data.loc[~imdb_data.loc[:,"review"].duplicated()]

In [6]:
tokenizer=ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [7]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(denoise_text)

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [8]:
#Stemming the text
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [9]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'nor', "won't", 'them', 'didn', 'did', 'those', 'so', "aren't", 'a', 'theirs', 'as', 'has', 'all', "needn't", 've', 'hadn', "hasn't", 'hers', 'above', 'off', 'that', 'before', 'an', 'd', 'wasn', 'do', 're', 'ma', 'out', 'o', 'your', 'after', "didn't", 'with', 'about', 'very', 'don', "mightn't", 'here', 'are', 'ours', 'can', "doesn't", 'own', 'which', "wasn't", 'yourselves', 'we', 'is', 'in', 'these', 'against', "hadn't", 'won', 'not', 'only', 'some', 'myself', 'when', 's', 'she', 'at', 'or', 'but', 'from', 'her', "weren't", 'yourself', 'you', 'same', 'haven', 'him', "isn't", 'for', 'itself', "wouldn't", 'such', "shan't", 'further', 'by', 'doing', 'because', 'if', 'more', "shouldn't", 'between', 'into', 't', 'himself', 'i', 'having', 'under', 'isn', 'once', 'too', 'themselves', 'shouldn', 'will', 'hasn', 'while', 'our', 'whom', 'most', 'why', 'shan', 'who', "it's", 'then', 'y', 'over', 'herself', 'just', 'were', 'my', 'll', 'had', 'to', 'have', 'until', 'couldn', 'ourselves', "don't", 

In [10]:
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stop)

In [11]:
X = vectorizer.fit_transform(imdb_data.review)
y = imdb_data.sentiment
pickle.dump(vectorizer, open('tranform.pkl', 'wb'))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB()

In [14]:
accuracy_score(y_test,clf.predict(X_test))*100

86.05405405405405

In [15]:
temp = np.array(["best movie ever"])
temp = vectorizer.transform(temp)

if clf.predict(temp)[0]:
    print('positive comment')
else:
    print('negeative comment')

positive comment


In [16]:
temp = np.array(["worst movie ever"])
temp = vectorizer.transform(temp)

if clf.predict(temp)[0]:
    print('positive comment')
else:
    print('negeative comment')

negeative comment


In [17]:
filename = 'nlp_model.pkl'
pickle.dump(clf, open(filename, 'wb'))